In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from deepchecks.tabular import Dataset
from deepchecks.tabular.suites import train_test_validation
from deepchecks.tabular.suites import model_evaluation



In [3]:
Xtrain = pd.read_csv('../train.csv')
Xtest = pd.read_csv('../holdout.csv')

In [4]:
Xtest.index = max(Xtrain.index)+1+np.arange(Xtest.shape[0])

In [5]:
Xtest

,age,work_class,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
17371,41,Private,High School grad,Separated,Service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
17372,44,Private,Masters,Never-Married,Professional,Not-in-family,Asian-Pac-Islander,Female,0.0,0.0,40.0,SE-Asia,<=50K
17373,53,Self-emp-inc,Doctorate,Separated,White-Collar,Not-in-family,White,Male,0.0,0.0,60.0,United-States,>50K
17374,46,Self-emp-not-inc,Bachelors,Never-Married,Blue-Collar,Not-in-family,White,Female,0.0,0.0,40.0,United-States,<=50K
17375,35,Private,High School grad,Never-Married,Blue-Collar,Unmarried,White,Male,0.0,0.0,40.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26414,32,Private,Bachelors,Never-Married,Sales,Own-child,White,Male,13550.0,0.0,35.0,United-States,>50K
26415,35,Private,Bachelors,Married,Sales,Husband,White,Male,7298.0,0.0,48.0,United-States,>50K
26416,42,Private,High School grad,Married,Admin,Wife,White,Female,0.0,0.0,36.0,United-States,>50K
26417,41,Private,Bachelors,Separated,Admin,Unmarried,White,Female,0.0,0.0,33.0,United-States,<=50K


In [6]:
cat_feat = [i for i in Xtrain.columns if Xtrain[i].dtype=='object']
cat_feat

['work_class',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native_country',
 'income']

In [7]:
cat_feat.remove('income')
clf = CatBoostClassifier(cat_features=cat_feat, 
                         n_estimators=100, 
                         verbose=False).fit(Xtrain.drop(['income'],axis=1),
                                            Xtrain['income'])

In [8]:
print(classification_report(Xtest['income'],clf.predict(Xtest.drop(['income'],axis=1))))

              precision    recall  f1-score   support

       <=50K       0.89      0.94      0.91      6833
        >50K       0.77      0.64      0.70      2215

    accuracy                           0.86      9048
   macro avg       0.83      0.79      0.81      9048
weighted avg       0.86      0.86      0.86      9048



In [9]:
train_ds1 = Dataset(Xtrain, label='income',cat_features=cat_feat, set_index_from_dataframe_index=True)
test_ds1 = Dataset(Xtest, label='income',cat_features=cat_feat, set_index_from_dataframe_index=True)

In [10]:
validation_suite1 = train_test_validation()
suite_result1 = validation_suite1.run(train_ds1, test_ds1)


In [11]:
suite_result1

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_7H4AC0EAJUONY1LNRTC4CQH07">Train Test Validat…

In [12]:
suite_result1.save_as_html('report.html')

'report.html'

In [13]:
suite_result1.show()

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_7BXZ1BHCFTWJMGI98FEADAAB7">Train Test Validat…

In [14]:
result1 = model_evaluation().run(train_dataset=train_ds1, test_dataset=test_ds1, model=clf)

In [15]:
result1.show()

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_NA68H9PHJPBG7C0BPY3QIWEJL">Model Evaluation S…

In [20]:
result1.save_as_html('model_report.html')

'model_report.html'